# Using the Python `chi` library

## Getting started

Before you do anything, you should first set up a few project variables. This is so your script knows which project to operate on. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. You can also set the site you want to use via the `region_name` setting - this defaults to `CHI@UC`.

In [ ]:
import chi

chi.set('project_name', 'your_project') # Replace with your project name
chi.set('region_name', 'CHI@UC')        # Optional, defaults to 'CHI@UC'

## Launching a baremetal server

Starting a new baremetal server is easy to do.

In [ ]:
from chi.lease import Lease

# Create a lease for 1 node
lease = Lease(node_type='compute_skylake')
lease.wait() # Ensure lease has started

# Launch 1 server
server = lease.create_server(image='CC-CentOS7')
server.wait() # Ensure server has started

## Connecting to your server

By default, the server will only have a private IP assigned. In order to SSH to the server, you should assign a public floating IP. There are are limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

Once a public IP is assigned, you can set up a remote connection to the node.

In [ ]:
from chi.ssh import Remote

# Assign a public floating IP
server.associate_floating_ip()

# Create a remote connection
remote = Remote(server=server)

You can use `remote.run` to run commands on the server directly from your Notebook.
A similar command is `remote.sudo`, which runs a command with escalated privileges.

### Example command: get CPU usage

In [ ]:
# Run some commands on your remote server
cpu_usage = remote.run("grep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage}'", hide=True)

print('Current CPU usage is {}%'.format(float(cpu_usage.stdout)))

## Collect monitoring metrics

If you are using an image based on one of the Chameleon-provided base images (such as CC-CentOS7), your server will automatically register several monitoring metrics to a Gnocchi database. [Read the docs][1] to learn more about this.

There is a convenience Gnocchi client instance provided by the `chi` module. In fact, you can get an easy instance to most OpenStack clients this way.

[1]: https://chameleoncloud.readthedocs.io/en/latest/technical/metrics.html

In [ ]:
# Note: if your server just started, this might return no data. You should wait
# a bit for all the metrics to be registered in the system.
gnocchi = chi.gnocchi()
metric_name = 'load@load'
granularity = 60 # seconds
measures = gnocchi.metric.get_measures(metric_name, 
                                       resource_id=server.id, 
                                       granularity=granularity)

In [ ]:
# Plot using pandas
import pandas as pd
df = pd.DataFrame(data=measures, columns=('timestamp', 'granularity', 'value'))
df = df.set_index('timestamp').drop(columns='granularity')

df.plot()